In [ ]:
import keras 
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense ,Dropout, MaxPooling2D ,Conv2D ,Flatten

(x_train,y_train),(x_test,y_test)=cifar10.load_data()

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(20,5))
for i in range(36):
    ax = fig.add_subplot(3, 12, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_train[i]))


x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255

from keras import utils

num_classes=len(np.unique(y_train))

y_train=utils.to_categorical(y_train,num_classes)

y_test=utils.to_categorical(y_test,num_classes)

(x_train,x_valid)=x_train[5000:],x_train[:5000]

(y_train,y_valid)=y_train[5000:],y_train[:5000]



model=Sequential()
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=(32,32,3)))
model.add(Conv2D(32,(3,3),padding='same',activation='relu'))
model.add(Conv2D(32,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv2D(64,(2,2),padding='same',activation='relu'))
model.add(Conv2D(64,(2,2),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.15))
model.add(Conv2D(128,(2,2),padding='same',activation='relu'))
model.add(Conv2D(128,(2,2),padding='same',activation='relu'))
model.add(Conv2D(128,(2,2),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10,activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adadelta', 
                  metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint

checkpoint=ModelCheckpoint(filepath='cnn_model.hdf5',verbose=1,save_best_only=True)

hist=model.fit(x_train,y_train,batch_size=64,epochs=100,verbose=2,callbacks=[checkpoint],validation_data=(x_valid,y_valid),shuffle=True)

model.load_weights('cnn_model.hdf5')

score=model.evaluate(x_test,y_test,verbose=0)
print(score[1])

y_hat = model.predict(x_test)

# define text labels (source: https://www.cs.toronto.edu/~kriz/cifar.html)
cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

fig = plt.figure(figsize=(20, 8))
for i, idx in enumerate(np.random.choice(x_test.shape[0], size=32, replace=False)):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx = np.argmax(y_hat[idx])
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(cifar10_labels[pred_idx], cifar10_labels[true_idx]),
                 color=("green" if pred_idx == true_idx else "red"))
